In [1]:
# Run this cell to install these packages
! pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set_context("talk")

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# delete this for final version
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
# delete this for final version

# uncomment this for final version
# import plotly.offline as py
# py.init_notebook_mode(connected=False)
# import plotly.graph_objs as go
# import plotly.figure_factory as ff
# uncomment this for final version

#import cufflinks as cf
#cf.set_config_file(offline=False, world_readable=True, theme='ggplot')

---

## 1. Section Name  <a id='section1'></a>

1. Understand the basic structure of this land use regression data set
    1. Predictors: 
        1. Remote sensing estimates of NO2 concentrations
        2. Land use characteristics like roads, impervious surfaces
        3. Population density
    2. Response variable: directly measured NO2 concentrations.

In [3]:
df_final = pd.read_csv('BechleLUR_2006_finalmodel.csv')

In [ ]:
df_final.columns

1. How to run multiple linear regression with scikit learn and interpret coefficients
    1. Here we'll just run the basic regression specification in Novotny et al, using their "BechleLUR_2006_finalmodel.csv" data
    2. Examine the model coefficients.  What are their units?  State in words what they represent.  Discuss their confidence intervals.  
    3. Side note: keep in mind that some variables will not go into the regression, specifically Latitude, Longitude, State and Predicted_NO2_ppb
        1. Perhaps ask the students why these shouldn't be included. 

In [ ]:
#plt.plot(df_final['WRF+DOMINO'], df_final['Observed_NO2_ppb'] - df_final['Predicted_NO2_ppb'],'.')

train_points = go.Scatter(name = "LUR Data", 
                          x = df_final['WRF+DOMINO'], y = df_final['Observed_NO2_ppb'], 
                          mode = 'markers')
# layout = go.Layout(autosize=False, width=800, height=600)


# uncomment this for final version
# py.iplot(go.Figure(data=[train_points]))
# uncomment this for final version

# delete this for final version
py.sign_in('ERG190Cstaff', '98mJNfe7TbJElrNNASOA')
py.iplot(go.Figure(data=[train_points]), filename='hw6')
# delete this for final version

In [ ]:
# Fit the model to the data
line_reg = linear_model.LinearRegression(fit_intercept=True)
predictors = df_final.loc[:,['WRF+DOMINO']]#:'Distance_to_coast_km']
output = df_final['Observed_NO2_ppb']
line_reg.fit(predictors, output)
# very important note: scikit-learn wants a data frame for the predictors -- if you have only one predictor and you pass in a pandas series, scikit-learn throws an error. 
np.hstack([line_reg.coef_, line_reg.intercept_]) 

In [ ]:
y_hat = line_reg.predict(predictors)
y_hat[:5]

In [ ]:
X_query = pd.DataFrame(np.linspace(predictors.min()-1, predictors.max() +1, 1000))
X_query.head()

In [ ]:
y_query_pred = line_reg.predict(X_query)
y_query_pred[:5]

---

## 2. Section Name  <a id='section2'></a>

2. How to interpret p-values and the pitfalls of p-hacking

---

## 3. The Akaike Information Criterion (NEED TO CLEAN UP + ADD QUESTIONS)  <a id='section3'></a>

3. How to use a model selection criterion like AIC.
    1. Here I'd like to have students add features from Novotny's "BechleLUR_2006_allmodelbuildingdata.csv" data set 
    2. Have them evaluate whether the new model with new features is a better model than the first one.  
    3. Don't have them work with all features from the allmodelbuildingdata set, instead pick a few features to give to the students.  Suggested:
        1. Population_800
        2. Major_400, Major_1200 (these will be strongly collinear with Major_800, which is in the final model)
        3. Others, if you wish.

Sources:

https://en.wikipedia.org/wiki/Akaike_information_criterion - Gives an okay summary of what AIC does at the level of this assignment
    
https://link.springer.com/book/10.1007%2F978-1-4612-1694-0 , pg. 215. The first paper that introduced and defined AIC. The level is beyond the scope of the course.

http://rlhick.people.wm.edu/posts/estimating-custom-mle.html - Estimiating maximum likelihoods. Give code in how to find maximum likelihoods.

The Akaike Information Criterion ($\text{AIC}$) assess the ***quality*** of a model given a set of data. Depending on the data that we use in our model, in this case, the features we add, AIC may be used to tell us how our model performs with the data given. Sometimes adding more data (features) improves the quality, sometimes less. Other times adding the right features may improve the quality.

$\text{AIC}$ is important because we can use it as a form of model selection. **Our goal is to find a model that has the highest *quality* given a list of models.** The higher the quality, the better our model performs and the more desirable it is. Your job in this section is to add features to *final_model* from *allmodelbulidingdata* and assess whether adding specific features improves the model or not. This may seem daunting, but we'll guide you in this process.

Now that you have some information about what AIC is doing and why it's important, we define $\text{AIC}$ as the following:

$\text{AIC} = 2 \times (\text{number of features}) - 2 \times \log(\text{maximum value of likelihood function})$

where $\log$ is $\ln$.  The smaller $\text{AIC}$ is, the greater the model performs. A likelihood function is a statistical topic that we won't go into, but we'll provide the code on how to implement it.

The way to interpret an $\text{AIC}$ number is that smaller it is, the greater the model performs.

First let's import what we need for AIC. We will need our our other data set and a function to produce our maximum value of our liklihood function:

In [ ]:
#Import to find maximum value of log likelihood
import statsmodels.regression.linear_model as sm

#Reading BechleLUR_2006_allmodelbuildingdata.csv into pandas
df_all = pd.read_csv('BechleLUR_2006_allmodelbuildingdata.csv')

#Cleaning data (happening early in the graph)
df_final_clean = df_final.drop(columns = 
                               ["Latitude", "Longitude", "State", "Predicted_NO2_ppb", 
                                "Location_type", "Monitor_ID"])


#Add column of zeros to adjust for constants
#df_final_clean["Constant"] = np.ones(len(df_final_clean))

#Optional: Write a function aic that takes in features and the maximum value of the 
#likelihood function and returns the aic.
###Solutions:
def aic(features, log_lik):
       return 2*(features - log_lik)
###

    
#Function to find log likelihood and number of features
def likandfeatures(X, y):
    '''
    Description: A function created to find the maximum value of the log-likelihood function in an OLS
    model. Also returns the number of features in the likelihood
    
    Inputs: X, An NxN pandas dataframe that contains your features
            y, An Nx1 pandas dataframe that you're trying to fit with X
            
    Returns: A tuple containing:
               The amount of features in X when finding the log-likelihood function
               The maximum value of the log-likelihood function
    '''
    #Adding a column (feature) of ones to consider constants because
    #Statsmodel does not support constants, so we must manually add one.
    #Thus, features = number of columns in X + 1
    X_final = X.assign(Constant = np.ones(len(y)))
    
    return (sm.OLS(y, X_final).fit().llf, len(X_final.columns))


#Running Model. This finds the Log-Likelihood function
log_L = sm.OLS(df_all["Population_800"], df_final_clean).fit().llf
log_L
#-3270.3083800895165

Test runs with features Duncan gave us

In [ ]:
tester = likandfeatures(df_final_clean, df_all['Population_800'])
print("This is the log-likelihood max value " + str(tester[1]))
print("This is the aic: " + str(aic(tester[0], tester[1])))
sm.OLS(df_all["Population_800"], df_final_clean.assign(Constant = np.ones(len(df_all)))).fit().summary()

In [ ]:
likandfeatures(df_final_clean, df_all["Major_400"])
tester = likandfeatures(df_final_clean, df_all["Major_400"])
print("This is the log-likelihood max value " + str(tester))
print("This is the aic: " + str(aic(len(df_final_clean.columns), tester)))
sm.OLS(df_all["Major_400"], df_final_clean.assign(Constant = np.ones(len(df_all)))).fit().summary()

In [ ]:
likandfeatures(df_final_clean, df_all["Major_1200"])
tester = likandfeatures(df_final_clean, df_all["Major_1200"])
print("This is the log-likelihood max value " + str(tester))
print("This is the aic: " + str(aic(len(df_final_clean.columns), tester)))
sm.OLS(df_all["Major_1200"], df_final_clean.assign(Constant = np.ones(len(df_all)))).fit().summary()

---

## 4. Section Name  <a id='section4'></a>

4. Address some of the potential problems that arise in linear regression models, including
    1. Non-linearity of the response-predictor relationships. 
    2. Correlation of error terms. 
    3. Non-constant variance of error terms.
    4. Outliers.
    5. High-leverage points.
    6. Collinearity.

For the "potential problems" learning objective, *Collinearity* is definitely an issue with the "allmodelbuildingdata" data set, so you could focus there.  *Correlation of error terms* and *Non-constant variance of error terms* are a little too hard to evaluate in this spatial setting.  I don't think there are any *Outliers* or *High-leverage points*.  And I'm not sure about *Non-linearity of the response-predictor relationships*.

---

## Bibliography

- Author - How entry was used. Link to website

---
Notebook developed by: Joshua Asuncion, Alex McMurry, Kevin Marroquin

Data Science Modules: http://data.berkeley.edu/education/modules
